In [1]:
# import dependencies
import os
import csv
from pandas import DataFrame
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
import glob

## Meters Inventory File

In [3]:
# display column names in a csv  
path = '../data_collection_munging/meters_inventory/meters_inventory.csv'
meters_inventory = pd.read_csv(path, sep=",")
print(len(meters_inventory))
meters_inventory.head(2)

33989


,address,latitude,longitude,meter_type,parking_policy,policy_complexity,rate_range,rate_type,meter_id,street_cleaning,park_limit
0,501 UNION DR,34.057219,-118.268751,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,WU150,12P-3P Mon,2.0
1,12200 VENICE BL,34.004300,-118.432007,Single-Space,8A-8P Mon-Sat,13,$1.00,FLAT,MV161,NaN,2.0


## Parking Citations File - - - with Lat/Lon coordinates

In [5]:
# load data 
path = '../data_collection_munging/parking_citations/data/citations_db_ready/citations_0000000001.csv'
headers_list = pd.read_csv(path, sep=",")
headers_list.head(2)

,agency,body_style,color,fine_amount,issue_date,latitude,location,longitude,make,meter_id,route,ticket_number,violation_code,violation_description
0,2,PA,BK,58.0,2015-12-21,33.945212,200 WORLD WAY,-118.401151,NISS,NaN,2R2,1104803000,8939,WHITE CURB
1,2,PA,WH,NaN,2015-12-26,33.945212,100 WORLD WAY,-118.401009,ACUR,NaN,2F11,1104820732,000,17104h


In [6]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/citations_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 4, 6, 9, 11, 13])
    li.append(df)
citations_v1 = pd.concat(li)
print(f'There are: {len(citations_v1)} records in dataframe')
citations_v1.head()

There are: 8076329 records in dataframe


,fine_amount,issue_date,location,meter_id,ticket_number,violation_description
0,58.0,2015-12-21,200 WORLD WAY,NaN,1104803000,WHITE CURB
1,NaN,2015-12-26,100 WORLD WAY,NaN,1104820732,17104h
2,93.0,2015-12-21,10635 FRIAR ST,NaN,1108321723,RED ZONE
3,363.0,2015-12-19,14006 RIVERSIDE DR,NaN,1109139006,HANDICAP/NO DP ID
4,68.0,2015-12-26,3200 CANYON DR,NaN,1109455266,PK IN PROH AREA


In [7]:
# convert "issue_date" column to a date 
citations_v1['issue_date'] = pd.to_datetime(citations_v1['issue_date'], dayfirst=True)

In [8]:
# preview the missing values and the % of missing values in each column
df = citations_v1
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 6 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
meter_id,5758449,71.3
fine_amount,3032,0.0
violation_description,390,0.0
issue_date,2,0.0
location,2,0.0


In [9]:
meters_lat_long_v1 = citations_v1.dropna(axis = 0, how ='any')

In [10]:
print(len(meters_lat_long_v1))

2317860


## Parking Citations File - - - without Lat/Lon coordinates

In [11]:
# display column names in a csv  
path = '../data_collection_munging/parking_citations/data/missed_lat_long_db_ready/missed_lat_long_0000000001.csv'
headers_list = pd.read_csv(path, sep=",")
headers_list.head(2)

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,agency,body_style,color,fine_amount,issue_date,issue_time,latitude,location,longitude,make,meter_id,route,ticket_number,violation_code,violation_description
0,1.0,PA,GY,50.0,2015-12-21,1251.0,99999,13147 WELBY WAY,99999,HOND,NaN,01521,1103341116,4000A1,NO EVIDENCE OF REG
1,1.0,VN,WH,50.0,2015-12-21,1435.0,99999,525 S MAIN ST,99999,GMC,NaN,1C51,1103700150,4000A1,NO EVIDENCE OF REG


In [12]:
# loop over csv files and extract columns needed

path = '../data_collection_munging/parking_citations/data/missed_lat_long_db_ready/'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, sep=",", usecols=[3, 4, 7, 10, 12, 14])
    li.append(df)
citations_v2 = pd.concat(li)
print(f'There are: {len(citations_v2)} records in dataframe')
citations_v2.head()

There are: 1401198 records in dataframe


,fine_amount,issue_date,location,meter_id,ticket_number,violation_description
0,50.0,2015-12-21,13147 WELBY WAY,NaN,1103341116,NO EVIDENCE OF REG
1,50.0,2015-12-21,525 S MAIN ST,NaN,1103700150,NO EVIDENCE OF REG
2,93.0,2015-09-15,GEORGIA ST/OLYMPIC,NaN,1105461453,NO STOPPING/STANDING
3,50.0,2015-09-15,SAN PEDRO S/O BOYD,NaN,1106226590,NO EVIDENCE OF REG
4,163.0,2015-12-17,SUNSET/ALVARADO,NaN,1106500452,PARK IN GRID LOCK ZN


In [13]:
# preview the missing values and the % of missing values in each column
df = citations_v2
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

missing_values_table(df)

Selected dataframe has 6 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
meter_id,1274696,91.0
fine_amount,4096,0.3
location,936,0.1
violation_description,621,0.0
issue_date,566,0.0


In [14]:
meters_v2 = citations_v2.dropna(axis = 0, how ='any')

In [15]:
# convert "issue_date" column to a date 
meters_v2['issue_date'] = pd.to_datetime(meters_v2['issue_date'], dayfirst=True)

/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
print(len(meters_v2))

126453


## Concatenate dataframes (appending rows with Meter IDs only)

In [17]:
df_all_rows = pd.concat([meters_lat_long_v1, meters_v2])
print(len(df_all_rows))

2444313


In [18]:
# reset index
df_all_rows=df_all_rows.reset_index(drop=True)
df_all_rows.head()

,fine_amount,issue_date,location,meter_id,ticket_number,violation_description
0,63.0,2015-12-28,4500 W AMBROSE AV,VH220,1112718014,METER EXPIRED
1,63.0,2015-12-28,4635 W HOLLYWOOD BL,VH14,1112718062,METER EXPIRED
2,68.0,2015-12-27,1428 ROXBURY DR,25,1113965311,PREF PARKING
3,68.0,2015-12-27,1488 EDRIS DR,25,1113965322,PREF PARKING
4,68.0,2015-12-27,1469 EDRIS DR,25,1113965333,PREF PARKING


In [19]:
# put year, month, and a day into separate columns
df_all_rows['Year'] = df_all_rows['issue_date'].dt.year
df_all_rows['Month'] = df_all_rows['issue_date'].dt.month
df_all_rows.drop(["issue_date"], axis = 1, inplace = True)

df_all_rows.head()

,fine_amount,location,meter_id,ticket_number,violation_description,Year,Month
0,63.0,4500 W AMBROSE AV,VH220,1112718014,METER EXPIRED,2015,12
1,63.0,4635 W HOLLYWOOD BL,VH14,1112718062,METER EXPIRED,2015,12
2,68.0,1428 ROXBURY DR,25,1113965311,PREF PARKING,2015,12
3,68.0,1488 EDRIS DR,25,1113965322,PREF PARKING,2015,12
4,68.0,1469 EDRIS DR,25,1113965333,PREF PARKING,2015,12


## Aggregate Citations by Meter ID and YY/MM

In [20]:
# count citations by year and month
df_top_frequency = df_all_rows.groupby(['Year', 'Month', 'violation_description', 'fine_amount', 'meter_id'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency.head()


/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Year,Month,violation_description,fine_amount,meter_id,counts
0,2015,8,PREFERENTIAL PARKING,68.0,37,1344
1,2016,8,PREFERENTIAL PARKING,68.0,37,1257
2,2016,10,PREFERENTIAL PARKING,68.0,37,1248
3,2016,6,PREFERENTIAL PARKING,68.0,37,1208
4,2015,6,PREFERENTIAL PARKING,68.0,37,1207


In [21]:
df_top_frequency['revenue'] = df_top_frequency.fine_amount * df_top_frequency.counts
print(len(df_top_frequency))
df_top_frequency.head()

898104


,Year,Month,violation_description,fine_amount,meter_id,counts,revenue
0,2015,8,PREFERENTIAL PARKING,68.0,37,1344,91392.0
1,2016,8,PREFERENTIAL PARKING,68.0,37,1257,85476.0
2,2016,10,PREFERENTIAL PARKING,68.0,37,1248,84864.0
3,2016,6,PREFERENTIAL PARKING,68.0,37,1208,82144.0
4,2015,6,PREFERENTIAL PARKING,68.0,37,1207,82076.0


In [22]:
# quick check of revenue per year
yy = df_top_frequency.groupby('Year')['revenue'].sum()
yy

Year
2011         146.0
2012          58.0
2013        1033.0
2014      787694.0
2015    40335662.0
2016    37885797.0
2017    35787314.0
2018    30077742.0
2019    12018822.0
Name: revenue, dtype: float64

In [23]:
# total revenue for the dataset
df_top_frequency['revenue'].sum()

156894268.0

## IMPORTANT

- There are 75 different types of violations at the parking meter. <br><br>
- For our machine learning model we are only interested in: <br>
    - meter expired (staying beyond the amount of prepaid time)
    - car parked outside meter lines (parking beyond meter lines)
    - overtime parking (staying beyond the posted time limit)<br><br>
    
- These 3 violations have the most parking tickets and we were able to cross reference them in meters inventory dataset.<br>

> Nevertheless, for our revenue analysis we are going to use full dataset.
    

In [24]:
print(len(df_top_frequency['violation_description'].unique()))

75


In [27]:
# save dataframe to csv file
df_top_frequency.to_csv("../analytics_visualizations/tableau_data/revenue_yy_mm.csv", encoding = 'utf-8', index = False)



## Aggregate Citations by Meter ID ONLY

In [28]:
# count citations by year and month
df_top_frequency_v2 = df_all_rows.groupby(['meter_id', 'location', 'violation_description', 'fine_amount'])['ticket_number'].agg(
    {"counts": len}).sort_values("counts", ascending=False).reset_index()

df_top_frequency_v2


/Users/pavelaltukhov/anaconda3/envs/meters_ml_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,meter_id,location,violation_description,fine_amount,counts
0,37,7000 HAWTHORN AVE,PREFERENTIAL PARKING,68.0,3981
1,49,2110 CORINTH AVE,PREFERENTIAL PARKING,68.0,3835
2,80101,1235 FIGUEROA PL,OFF STR/OVERTIME/MTR,58.0,3765
3,SA16620,1152 LEMOYNE ST,OFF STR/OVERTIME/MTR,58.0,3055
4,PW17075,2377 MIDVALE AVE,OFF STR/OVERTIME/MTR,58.0,2572
5,WP8204,20 WASHINGTON BLVD E,METER EXP.,63.0,2513
6,NH16276,11231 MAGNOLIA BLVD,OFF STR/OVERTIME/MTR,58.0,2364
7,SO17626,15314 DICKENS ST W,OFF STR/OVERTIME/MTR,58.0,2346
8,37,1665 SYCAMORE AVE N,PREFERENTIAL PARKING,68.0,2346
9,37,1600 ORANGE DR N,PREFERENTIAL PARKING,68.0,2130


In [30]:
df_top_frequency_v2['revenue'] = df_top_frequency_v2.fine_amount * df_top_frequency_v2.counts
df_top_frequency_v2.head()

,meter_id,location,violation_description,fine_amount,counts,revenue
0,37,7000 HAWTHORN AVE,PREFERENTIAL PARKING,68.0,3981,270708.0
1,49,2110 CORINTH AVE,PREFERENTIAL PARKING,68.0,3835,260780.0
2,80101,1235 FIGUEROA PL,OFF STR/OVERTIME/MTR,58.0,3765,218370.0
3,SA16620,1152 LEMOYNE ST,OFF STR/OVERTIME/MTR,58.0,3055,177190.0
4,PW17075,2377 MIDVALE AVE,OFF STR/OVERTIME/MTR,58.0,2572,149176.0


## Keep Meters with >10 Parking Citations

In [31]:
df_top_frequency_1 = df_top_frequency_v2[df_top_frequency_v2.counts > 10]


print(f'UNIQUE METERS: {len(df_top_frequency_v2)}')
print("=============================================")
print(f'METERS WITH <10 TICKETS: {len(df_top_frequency_v2)-len(df_top_frequency_1)}')
print("=============================================")
print(f'METERS FOR FURTHER PROCESSING: {len(df_top_frequency_1)}')

UNIQUE METERS: 264134
METERS WITH <10 TICKETS: 218922
METERS FOR FURTHER PROCESSING: 45212


In [32]:
df_top_frequency_1['revenue'].sum()

123762514.0

In [34]:
df_top_frequency_1.to_csv("../analytics_visualizations/tableau_data/revenue_meter_id.csv", encoding = 'utf-8', index = False)



## Concatenate Meters Inventory with Revenue per Meter dataframes

In [35]:
ml_df = pd.merge(df_top_frequency_1, meters_inventory, how='inner', on = 'meter_id')

In [36]:
ml_df

,meter_id,location,violation_description,fine_amount,counts,revenue,address,latitude,longitude,meter_type,parking_policy,policy_complexity,rate_range,rate_type,street_cleaning,park_limit
0,V77,101 VENICE WAY S,METER EXP.,63.0,465,29295.0,101 VENICE WAY (S),33.987418,-118.471119,Single-Space,8A-6P Daily,11,$2.00,FLAT,12P-2P Mon,1.000000
1,V75,100 VENICE WAY,METER EXP.,63.0,450,28350.0,100 VENICE WAY (S),33.987243,-118.471043,Single-Space,8A-6P Daily,11,$2.00,FLAT,12P-2P Tue,1.000000
2,NH940,4301 TUJUNGA AV,METER EXP.,63.0,447,28161.0,4301 TUJUNGA AV,34.148960,-118.379082,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,NaN,1.000000
3,NH935,4301 TUJUNGA AV,METER EXP.,63.0,446,28098.0,4301 TUJUNGA AV,34.149560,-118.379083,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,NaN,1.000000
4,V73,100 VENICE WAY,METER EXP.,63.0,430,27090.0,100 VENICE WAY (S),33.987264,-118.471196,Single-Space,8A-6P Daily,11,$2.00,FLAT,12P-2P Tue,1.000000
5,NH934,4301 TUJUNGA AV,METER EXP.,63.0,427,26901.0,4301 TUJUNGA AV,34.149693,-118.379082,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,NaN,1.000000
6,V74,100 VENICE WAY,METER EXP.,63.0,420,26460.0,100 VENICE WAY (S),33.987256,-118.471134,Single-Space,8A-6P Daily,11,$2.00,FLAT,12P-2P Tue,1.000000
7,V74,100 VENICE WAY,OUTSIDE LINES/METER,58.0,25,1450.0,100 VENICE WAY (S),33.987256,-118.471134,Single-Space,8A-6P Daily,11,$2.00,FLAT,12P-2P Tue,1.000000
8,NH936,4301 TUJUNGA AV,METER EXP.,63.0,415,26145.0,4301 TUJUNGA AV,34.149426,-118.379082,Single-Space,8A-6P Mon-Sat,13,$1.00,FLAT,NaN,1.000000
9,CB156,1300 HOPE ST S,METER EXP.,63.0,415,26145.0,1400 S HOPE ST,34.037869,-118.266592,Single-Space,8A-8P Mon-Sat,13,$1 - $2,TOD,NaN,4.000000


In [37]:
ml_df['revenue'].sum()

82010231.0

In [38]:
ml_df['violation_description'].unique()
# print(len(ml_df['meter_type'].unique()))

array(['METER EXP.', 'OUTSIDE LINES/METER', 'OFF STR/OVERTIME/MTR',
       'METER EXPIRED'], dtype=object)

In [39]:
#standardize the name of the violation description column
d = {'METER EXP.': 'METER EXPIRED'}
ml_df = ml_df.replace(d)

In [40]:
#drop redundant columns
ml_df.drop(ml_df.columns[[6, 10, 12, 14]], axis = 1, inplace = True)
ml_df.columns

Index(['meter_id', 'location', 'violation_description', 'fine_amount',
       'counts', 'revenue', 'latitude', 'longitude', 'meter_type',
       'policy_complexity', 'rate_type', 'park_limit'],
      dtype='object')

In [41]:
ml_df.to_csv("ml_model_data/meters_for_ml.csv", encoding = 'utf-8', index = False)